In [ ]:
import time
from random import random
from selenium import webdriver
from logging import raiseExceptions
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
service = Service()
options = webdriver.ChromeOptions()
options.add_extension("adblock.crx")
options.add_argument("--disable-application-cache")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-setuid-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.page_load_strategy = 'eager'
prefs = {
    "profile.managed_default_content_settings.images": 2,
    "profile.managed_default_content_settings.stylesheets": 2,
    "profile.managed_default_content_settings.plugins": 2,
    "profile.managed_default_content_settings.popups": 2,
    "profile.managed_default_content_settings.geolocation": 2,
    "profile.managed_default_content_settings.notifications": 2,
    "profile.managed_default_content_settings.media_stream": 2,
    "profile.managed_default_content_settings.javascript": 1,
    "profile.managed_default_content_settings.frame": 2
}

options.add_experimental_option("prefs", prefs)
options.add_experimental_option("prefs", prefs)
options.set_capability("pageLoadStrategy", "eager")

In [4]:
driver = webdriver.Chrome(service=service, options=options)

In [59]:
driver.get('https://www.tse.jus.br/comunicacao/noticias?year=2024&month=11&period=&formData=')

In [ ]:
news = driver.find_elements(By.CLASS_NAME, 'news-list-title')
while (len(news) <= 150):
    btnMore = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located(
            (By.ID,
                'news-list-read-more-button'
                )
        )
    )
    time.sleep(2)
    driver.execute_script("window.scrollBy(0, 10000);")
    while(btnMore.text != 'Veja outras notícias'):
        time.sleep(0.1)    
        btnMore = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located(
                (By.ID,
                    'news-list-read-more-button'
                    )
            )
        )
            
    btnMore.click()
    news = driver.find_elements(By.CLASS_NAME, 'news-list-title')

In [ ]:
list = ['#', '\'', '/', ',', '.', '', ']', '[', '(', ')', '*', '@', '!', '?', '|', ':', ';', '&', '–', '”', '“', '…', '=', ',', '.', '–', '"', '‘', '’', '(', ')', ]
trainNews = []
testNews = []
aux = []
n = 0

for card in news:
    text = ''
    n += 1

    #  Scroll to the card and click
    driver.execute_script("arguments[0].scrollIntoView({behavior:'smooth',block:'center'});", card)
    link = card.find_element(By.XPATH, ".//ancestor::a[1]").get_attribute("href")
    driver.execute_script(f"window.open('{link}', '_blank');")

    # Change the driver window
    driver.switch_to.window(driver.window_handles[-1])

    # Search all the news and store them
    try:
        timeout = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='master_main']"))
        )

        pNews = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, 'p'))
        )
        pNews = pNews[:-7]

        for p in pNews:
            text += (p.text.lstrip()).rstrip() + ' '
        
        text = text.split()
        print(n)

        aux = []
        if (random() < 0.8):
            for words in text:
                for char in list:
                    words = words.replace(char, '')
                aux.append(words.lower())

            aux = [word for word in aux if word != '']
            aux.insert(0, 'r')
            trainNews.append(aux)
        else:
            for words in text:
                for char in list:
                    words = words.replace(char, '')
                aux.append(words.lower())

            aux = [word for word in aux if word != '']
            aux.insert(0, 'r')
            testNews.append(aux)

    except Exception as e:
        print("error:", e)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        continue

    except TimeoutException as t:
        print("timeout", t)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        continue

    # Close the window
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

In [ ]:
arqOut = open('./TRAINrealnews.txt', 'w', encoding='utf-8')
for news in trainNews:
    print(news, file=arqOut)
arqOut.close()

In [ ]:
arqOut = open('./TESTrealnews.txt', 'w', encoding='utf-8')
for news in testNews:
    print(news, file=arqOut)
arqOut.close()